# Import module

The link to get [ImageAnalysis3](https://github.com/zhengpuas47/ImageAnalysis3) 

Or from the Zhuang lab archived [source_tools](https://github.com/ZhuangLab/Chromatin_Analysis_2020_cell/tree/master/sequential_tracing/source)

## ImageAnalysis3 and basic modules

In [1]:
%run "C:\Users\shiwei\Documents\ImageAnalysis3\required_files\Startup_py3.py"
sys.path.append(r"C:\Users\shiwei\Documents")

import ImageAnalysis3 as ia
from ImageAnalysis3 import *
from ImageAnalysis3.classes import _allowed_kwds

import h5py
import ast
import pandas as pd

print(os.getpid())

23588


## Chromatin_analysis_tools etc

See **functions** in the repository for [AnalysisTool_Chromatin](../../README.md)

In [2]:
# Chromatin_analysis_tools (ATC)
# Get path for the py containing functions
import os
import sys
import importlib
module_path =r'C:\Users\shiwei\Documents\AnalysisTool_Chromatin'
if module_path not in sys.path:
    sys.path.append(module_path)
    
# import relevant modules
import gene_selection 
importlib.reload(gene_selection)
import gene_to_loci
importlib.reload(gene_to_loci)
import gene_activity
importlib.reload(gene_activity)
import loci_1d_features
importlib.reload(loci_1d_features)  

import atac_to_loci
importlib.reload(atac_to_loci)

<module 'atac_to_loci' from 'C:\\Users\\shiwei\\Documents\\AnalysisTool_Chromatin\\atac_to_loci.py'>

# Define folders

In [3]:
# main folder for postanalysis
postanalysis_folder = r'L:\Shiwei\postanalysis_2024\v0'
# input files for postanalysis
input_folder = os.path.join(postanalysis_folder, 'resources_from_preprocess')

# output file to be generated
output_main_folder = os.path.join(postanalysis_folder, 'locus_annotation')

output_analysis_folder = os.path.join(output_main_folder, 'analysis')
output_figure_folder = os.path.join(output_main_folder, 'figures')

# make new folder if needed
make_output_folder = True

if make_output_folder and not os.path.exists(output_analysis_folder):
    os.makedirs(output_analysis_folder)
    print(f'Generating analysis folder: {output_analysis_folder}.')
elif os.path.exists(output_analysis_folder):
    print(f'Use existing analysis folder: {output_analysis_folder}.')
    
if make_output_folder and not os.path.exists(output_figure_folder):
    os.makedirs(output_figure_folder)
    print(f'Generating figure folder: {output_figure_folder}.')
elif os.path.exists(output_figure_folder):
    print(f'Use existing figure folder: {output_figure_folder}.')

Use existing analysis folder: L:\Shiwei\postanalysis_2024\v0\locus_annotation\analysis.
Use existing figure folder: L:\Shiwei\postanalysis_2024\v0\locus_annotation\figures.


# Plotting parameters

In [4]:
%matplotlib inline
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
import matplotlib.pyplot as plt
plt.rc('font', family='serif')
plt.rc('font', serif='Arial')

from ImageAnalysis3.figure_tools import _double_col_width, _single_col_width, _font_size, _ticklabel_size,_ticklabel_width

import seaborn as sns
sns.set_context("paper", rc={"font.size":_font_size,"axes.titlesize":_font_size+1,"axes.labelsize":_font_size})  

# Load data relevant information

## load codebook with peak annotation

annotated codebook can be generated similarly with ATAC data using the notebook below:


[external/scripts/pair_tag/2_adjacent_h3k27me3_peak_annotation_for_merfish_loci](../../../external/scripts/pair_tag/2_adjacent_h3k27me3_peak_annotation_for_merfish_loci.ipynb)

information related to ATAC can be found from the folder in the repository:
[ATACseq_MOp_folder](../../../external/scripts/sn_atac)

In [5]:
# load codebook
codebook_folder = output_analysis_folder

# Load codebook and sort
codebook_fname = os.path.join(codebook_folder,'MERFISH_loci_adjacent_peaks_center.csv')
codebook_df = pd.read_csv (codebook_fname, index_col=0)
codebook_df = loci_1d_features.sort_loci_df_by_chr_order (codebook_df)

codebook_df.head()

,name,id,chr,chr_order,library,adjacent_peaks_0kb_center,adjacent_peaks_50kb_center,adjacent_peaks_100kb_center,adjacent_peaks_500kb_center,adjacent_peaks_1000kb_center,adjacent_peaks_2000kb_center
loci_name,,,,,,,,,,,
chr1_3742742_3759944,1:3742742-3759944,1,1,0,CTP11,chr1_3745001_3750000; chr1_3750001_3755000; ch...,chr1_3695001_3700000; chr1_3700001_3705000; ch...,chr1_3645001_3650000; chr1_3650001_3655000; ch...,chr1_3245001_3250000; chr1_3250001_3255000; ch...,chr1_3000001_3005000; chr1_3005001_3010000; ch...,chr1_3000001_3005000; chr1_3005001_3010000; ch...
chr1_6245958_6258969,1:6245958-6258969,2,1,1,CTP11,chr1_6245001_6250000; chr1_6250001_6255000; ch...,chr1_6195001_6200000; chr1_6200001_6205000; ch...,chr1_6145001_6150000; chr1_6150001_6155000; ch...,chr1_5745001_5750000; chr1_5750001_5755000; ch...,chr1_5245001_5250000; chr1_5250001_5255000; ch...,chr1_4245001_4250000; chr1_4250001_4255000; ch...
chr1_8740008_8759916,1:8740008-8759916,3,1,2,CTP11,chr1_8740001_8745000; chr1_8745001_8750000; ch...,chr1_8695001_8700000; chr1_8700001_8705000; ch...,chr1_8640001_8645000; chr1_8645001_8650000; ch...,chr1_8240001_8245000; chr1_8245001_8250000; ch...,chr1_7740001_7745000; chr1_7745001_7750000; ch...,chr1_6740001_6745000; chr1_6745001_6750000; ch...
chr1_9627926_9637875,1:9627926-9637875,1,1,3,CTP13,chr1_9635001_9640000,chr1_9580001_9585000; chr1_9585001_9590000; ch...,chr1_9530001_9535000; chr1_9535001_9540000; ch...,chr1_9130001_9135000; chr1_9135001_9140000; ch...,chr1_8630001_8635000; chr1_8635001_8640000; ch...,chr1_7630001_7635000; chr1_7635001_7640000; ch...
chr1_9799472_9811359,1:9799472-9811359,2,1,4,CTP13,chr1_9800001_9805000; chr1_9805001_9810000,chr1_9750001_9755000; chr1_9755001_9760000; ch...,chr1_9705001_9710000; chr1_9710001_9715000; ch...,chr1_9300001_9305000; chr1_9305001_9310000; ch...,chr1_8800001_8805000; chr1_8805001_8810000; ch...,chr1_7800001_7805000; chr1_7805001_7810000; ch...


## load analyzed ATACseq annData

Data can be generated from the notebook (after transfer the cell labels to filtered data):

[external/scripts/sn_atac/3_label_atac_anndata_mop_episcanpy](../../../external/scripts/sn_atac/3_label_atac_anndata_mop_episcanpy.ipynb)

In [6]:
# Get loaded adata from other notebook
import os
import scanpy as sc
# L drive is Crick Pu_SSD_0
scRNA_folder = r'L:\Shiwei\DNA_MERFISH_analysis\Analysis_MOp_methylation_ATAC\data\processed_2024'
# load from here for saved h5ad
#adata = sc.read(os.path.join(scRNA_folder,r'MOp_ATAC_combined_preprocessed.h5ad'))
adata = sc.read(os.path.join(scRNA_folder,r'MOp_ATAC_combined_labeled.h5ad'))

In [9]:
# inspect peak count of the h5ad
print(np.max(adata.layers['normalized']))
print(np.max(adata.layers['binary']))
print(np.max(adata.X))

# use all vars
adata_ori = adata.raw.to_adata()
print(np.max(adata_ori.X))

4.146
1.0
1.6382197
1.0


In [10]:
adata.obs.head()

,barcode,TN,UM,PP,UQ,CM,file,sample,nb_features,log_nb_features,cluster,leiden,subclass_label_new,class_label_new,neuron_identity
4,AGCGATAGAACCAGGTAGGATAACATAGAGGC,19152.0,17586.0,17545.0,8348.0,0.0,E:\DNA-MERFISH_analysis\Analysis_MOp_methylati...,atac_1,4047,3.609594,L5.IT.b,3,L5 IT,Gluta,Neuronal
5,AGCGATAGAACCAGGTATAGCCTTAGGCGAAG,14876.0,13837.0,13801.0,6583.0,0.0,E:\DNA-MERFISH_analysis\Analysis_MOp_methylati...,atac_1,3585,3.555094,L6.CT,18,L6 CT,Gluta,Neuronal
6,AGCGATAGAACCAGGTATAGCCTTATAGAGGC,28532.0,26303.0,26202.0,12437.0,0.0,E:\DNA-MERFISH_analysis\Analysis_MOp_methylati...,atac_1,8088,3.908646,L6.IT,7,L6 IT,Gluta,Neuronal
8,AGCGATAGAACCAGGTATAGCCTTGTACTGAC,46202.0,42880.0,42723.0,20345.0,0.0,E:\DNA-MERFISH_analysis\Analysis_MOp_methylati...,atac_1,11948,4.078094,L5.IT.b,7,L5 IT,Gluta,Neuronal
9,AGCGATAGAACCAGGTATTCGTTGTAATCTTA,3007.0,2655.0,2645.0,1341.0,0.0,E:\DNA-MERFISH_analysis\Analysis_MOp_methylati...,atac_1,729,2.863323,L23.a,21,L2/3 IT,Gluta,Neuronal


In [11]:
adata.var.head()

,name,transcript_annotation,n_cells,commonness,prop_shared_cells,variability_score
chrY_255001_260000,chrY:255001-260000,intergenic,138.0,138.0,0.001020,0.501020
chrY_805001_810000,chrY:805001-810000,intergenic,315.0,315.0,0.002328,0.502328
chrY_870001_875000,chrY:870001-875000,intergenic,818.0,818.0,0.006045,0.506045
chrY_895001_900000,chrY:895001-900000,intergenic,1203.0,1203.0,0.008890,0.508890
chrY_900001_905000,chrY:900001-905000,intergenic,203.0,203.0,0.001500,0.501500


# Extract peak numbers

In [12]:
# change index name as imaged loci to find nearby genes
import gene_to_loci as gl

imaged_loci_df = codebook_df.copy(deep=True)

imaged_loci_df.index.name = 'Imaged_loci'
imaged_loci_df = gl.direct_get_genes_near_gene_dataframe (imaged_loci_df,
                                   codebook_df, 
                                   adjacent_gene_col = None)


imaged_loci_df

Get all existing adjacent gene columns.


,name,id,chr,chr_order,library,adjacent_peaks_0kb_center,adjacent_peaks_50kb_center,adjacent_peaks_100kb_center,adjacent_peaks_500kb_center,adjacent_peaks_1000kb_center,adjacent_peaks_2000kb_center
Imaged_loci,,,,,,,,,,,
chr1_3742742_3759944,1:3742742-3759944,1,1,0,CTP11,chr1_3745001_3750000; chr1_3750001_3755000; ch...,chr1_3695001_3700000; chr1_3700001_3705000; ch...,chr1_3645001_3650000; chr1_3650001_3655000; ch...,chr1_3245001_3250000; chr1_3250001_3255000; ch...,chr1_3000001_3005000; chr1_3005001_3010000; ch...,chr1_3000001_3005000; chr1_3005001_3010000; ch...
chr1_6245958_6258969,1:6245958-6258969,2,1,1,CTP11,chr1_6245001_6250000; chr1_6250001_6255000; ch...,chr1_6195001_6200000; chr1_6200001_6205000; ch...,chr1_6145001_6150000; chr1_6150001_6155000; ch...,chr1_5745001_5750000; chr1_5750001_5755000; ch...,chr1_5245001_5250000; chr1_5250001_5255000; ch...,chr1_4245001_4250000; chr1_4250001_4255000; ch...
chr1_8740008_8759916,1:8740008-8759916,3,1,2,CTP11,chr1_8740001_8745000; chr1_8745001_8750000; ch...,chr1_8695001_8700000; chr1_8700001_8705000; ch...,chr1_8640001_8645000; chr1_8645001_8650000; ch...,chr1_8240001_8245000; chr1_8245001_8250000; ch...,chr1_7740001_7745000; chr1_7745001_7750000; ch...,chr1_6740001_6745000; chr1_6745001_6750000; ch...
chr1_9627926_9637875,1:9627926-9637875,1,1,3,CTP13,chr1_9635001_9640000,chr1_9580001_9585000; chr1_9585001_9590000; ch...,chr1_9530001_9535000; chr1_9535001_9540000; ch...,chr1_9130001_9135000; chr1_9135001_9140000; ch...,chr1_8630001_8635000; chr1_8635001_8640000; ch...,chr1_7630001_7635000; chr1_7635001_7640000; ch...
chr1_9799472_9811359,1:9799472-9811359,2,1,4,CTP13,chr1_9800001_9805000; chr1_9805001_9810000,chr1_9750001_9755000; chr1_9755001_9760000; ch...,chr1_9705001_9710000; chr1_9710001_9715000; ch...,chr1_9300001_9305000; chr1_9305001_9310000; ch...,chr1_8800001_8805000; chr1_8805001_8810000; ch...,chr1_7800001_7805000; chr1_7805001_7810000; ch...
...,...,...,...,...,...,...,...,...,...,...,...
chrX_166247682_166259932,X:166247682-166259932,1059,X,60,CTP11,chrX_166250001_166255000; chrX_166255001_16626...,chrX_166200001_166205000; chrX_166205001_16621...,chrX_166150001_166155000; chrX_166155001_16616...,chrX_165750001_165755000; chrX_165755001_16576...,chrX_165250001_165255000; chrX_165255001_16526...,chrX_164250001_164255000; chrX_164255001_16426...
chrX_167157164_167167452,X:167157164-167167452,990,X,61,CTP13,chrX_167155001_167160000; chrX_167160001_16716...,chrX_167105001_167110000; chrX_167110001_16711...,chrX_167055001_167060000; chrX_167060001_16706...,chrX_166655001_166660000; chrX_166665001_16667...,chrX_166155001_166160000; chrX_166160001_16616...,chrX_165155001_165160000; chrX_165160001_16516...
chrX_168746045_168757590,X:168746045-168757590,1060,X,62,CTP11,chrX_168745001_168750000; chrX_168750001_16875...,chrX_168695001_168700000; chrX_168700001_16870...,chrX_168645001_168650000; chrX_168650001_16865...,chrX_168245001_168250000; chrX_168250001_16825...,chrX_167745001_167750000; chrX_167750001_16775...,chrX_166745001_166750000; chrX_166750001_16675...


In [14]:
imaged_loci_df.columns

Index(['name', 'id', 'chr', 'chr_order', 'library',
       'adjacent_peaks_0kb_center', 'adjacent_peaks_50kb_center',
       'adjacent_peaks_100kb_center', 'adjacent_peaks_500kb_center',
       'adjacent_peaks_1000kb_center', 'adjacent_peaks_2000kb_center'],
      dtype='object')

## process for major classes

In [15]:
groupby_adata = 'class_label_new'

np.unique(list(adata_ori.obs[groupby_adata]))

array(['Astro', 'Endo', 'GABA', 'Gluta', 'Micro', 'Oligo', 'Other', 'SMC',
       'nan'], dtype='<U32')

In [18]:
# output_folder
output_folder = os.path.join(output_analysis_folder, r'10x_snATAC\10x_class')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print ('Generate output folder')

Generate output folder


In [19]:
%matplotlib inline

import gene_activity
import loci_1d_features
from scipy import stats
import seaborn as sns
from tqdm import tqdm

bin_size =2000 # extend both direction
adjcent_col = f'adjacent_peaks_{bin_size}kb_center'
activity_type = 'sum' # sum of all gene associated to a loci for each single cell

expression_res_df_dict = {}

expression_res_df_dict = {}
sel_class_to_process = [c for c in np.unique(list(adata_ori.obs[groupby_adata])) if c!='nan']

#simple loop
for _group in sel_class_to_process[:]:
    
    print (f'Process ATACseq data for {_group}')
    
    expression_res_dict={}
    sorted_group_order = [_group]
    
    _group_fname = _group.replace('/','_').replace(' ','_')

    imaged_loci_df_group = loci_1d_features.codebook_chr_order_for_loci_dataframe (imaged_loci_df, 
                                               codebook_df, 
                                               sel_cols =['chr','chr_order','id'], 
                                               sort_df = True,
                                               sort_by_chr=True)

    loci_key_list = loci_1d_features.sorted_loci_keys_for_loci_dataframe(imaged_loci_df_group)
    loci_ori_ind = loci_1d_features.find_chr_loci_iloc_from_loci_keys (codebook_df, loci_key_list)

    # for loci along the chromosome, append the measurements for each single cell
    for _ind, sel_loci_ind in tqdm(enumerate(imaged_loci_df_group.index.tolist()[:])):

        # get adjacent gene expression (which are peaks in this case)
        sel_genes=imaged_loci_df_group.loc[sel_loci_ind][adjcent_col].split('; ')
        sel_adata =  adata_ori[:,adata_ori.var.index.isin(sel_genes)]
        marker_expressions = gene_activity.gene_activity_raw_groups(sel_genes,
                            sel_adata, 
                            sorted_group_order,
                            groupby_adata,
                            ref_norm_list = [],
                            report_type =activity_type)

        expression_res_dict[_ind]=list(marker_expressions[_group])
        
    # convert dict to df as loci by cell
    expression_res_df = pd.DataFrame.from_dict(expression_res_dict, orient='index')
    expression_res_df['loci_name']=codebook_df.index.tolist()
    expression_res_df = expression_res_df.set_index ('loci_name')
    expression_res_df_dict[_group]=expression_res_df
    # save
    output_df_fname = os.path.join(output_folder, f'MERFISH_loci_10x_expression_2X_{bin_size}kb_for_{_group_fname}.csv')
    expression_res_df.to_csv(output_df_fname)
    print ('=========================================================')


Process RNAseq data for Astro


1982it [04:06,  8.04it/s]


Process RNAseq data for Endo


1982it [03:08, 10.51it/s]


Process RNAseq data for GABA


...

1982it [05:12,  6.35it/s]


Process RNAseq data for Other


1982it [03:20,  9.87it/s]


Process RNAseq data for SMC


1982it [03:09, 10.46it/s]


In [20]:
expression_res_df.head()

,0,1,2,3,4,5,6,7,8,9,...,539,540,541,542,543,544,545,546,547,548
loci_name,,,,,,,,,,,,,,,,,,,,,
chr1_3742742_3759944,7.0,7.0,7.0,0.0,2.0,3.0,0.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
chr1_6245958_6258969,5.0,8.0,5.0,0.0,1.0,0.0,0.0,2.0,1.0,2.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
chr1_8740008_8759916,16.0,13.0,7.0,0.0,1.0,2.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,4.0
chr1_9627926_9637875,18.0,10.0,8.0,1.0,1.0,3.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0
chr1_9799472_9811359,19.0,10.0,9.0,1.0,1.0,3.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0


## process for subclasses

In [21]:
groupby_adata = 'subclass_label_new'
#groupby_adata = 'cluster'

np.unique(list(adata_ori.obs[groupby_adata]))
sel_class_to_process = [c for c in np.unique(list(adata_ori.obs[groupby_adata])) if c!='nan']
sel_class_to_process

['Astro',
 'CGE',
 'Endo',
 'L2/3 IT',
 'L4/5 IT',
 'L5 ET',
 'L5 IT',
 'L5/6 NP',
 'L6 CT',
 'L6 IT',
 'Micro',
 'OPC',
 'Oligo',
 'Other',
 'Pvalb',
 'SMC',
 'Sst']

In [22]:
# output_folder
output_folder = os.path.join(output_analysis_folder, r'10x_snATAC\10x_subclass')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print ('Generate output folder')

In [23]:
%matplotlib inline

import gene_activity
import loci_1d_features
from scipy import stats
import seaborn as sns
from tqdm import tqdm

bin_size =2000 # extend both direction
adjcent_col = f'adjacent_peaks_{bin_size}kb_center'
activity_type = 'sum' # sum of all gene associated to a loci for each single cell

expression_res_df_dict = {}
sel_class_to_process = [c for c in np.unique(list(adata_ori.obs[groupby_adata])) if c!='nan']

# simple loop
for _group in sel_class_to_process[:]:
    _group_fname = _group.replace('/','_').replace(' ','_')
    
    print (f'Process ATACseq data for {_group}')
    
    expression_res_dict={}
    sorted_group_order = [_group]

    imaged_loci_df_group = loci_1d_features.codebook_chr_order_for_loci_dataframe (imaged_loci_df, 
                                               codebook_df, 
                                               sel_cols =['chr','chr_order','id'], 
                                               sort_df = True,
                                               sort_by_chr=True)

    loci_key_list = loci_1d_features.sorted_loci_keys_for_loci_dataframe(imaged_loci_df_group)
    loci_ori_ind = loci_1d_features.find_chr_loci_iloc_from_loci_keys (codebook_df, loci_key_list)

    # for loci along the chromosome, append the measurements for each single cell
    for _ind, sel_loci_ind in tqdm(enumerate(imaged_loci_df_group.index.tolist()[:])):

        # get adjacent gene expression (which are peaks in this case)
        sel_genes=imaged_loci_df_group.loc[sel_loci_ind][adjcent_col].split('; ')
        sel_adata =  adata_ori[:,adata_ori.var.index.isin(sel_genes)]
        marker_expressions = gene_activity.gene_activity_raw_groups(sel_genes,
                            sel_adata, 
                            sorted_group_order,
                            groupby_adata,
                            ref_norm_list = [],
                            report_type =activity_type)

        expression_res_dict[_ind]=list(marker_expressions[_group])
        
    # convert dict to df as loci by cell
    expression_res_df = pd.DataFrame.from_dict(expression_res_dict, orient='index')
    expression_res_df['loci_name']=codebook_df.index.tolist()
    expression_res_df = expression_res_df.set_index ('loci_name')
    expression_res_df_dict[_group]=expression_res_df
    # save
    output_df_fname = os.path.join(output_folder, f'MERFISH_loci_10x_ATAC_2X_{bin_size}kb_for_{_group_fname}.csv')
    expression_res_df.to_csv(output_df_fname)
    print ('=========================================================')


Process ATACseq data for Astro


1982it [04:22,  7.54it/s]


Process ATACseq data for CGE


1982it [03:41,  8.94it/s]


Process ATACseq data for Endo


...

1982it [03:34,  9.25it/s]


Process ATACseq data for SMC


1982it [03:09, 10.45it/s]


Process ATACseq data for Sst


1982it [03:23,  9.73it/s]


In [24]:
expression_res_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2459,2460,2461,2462,2463,2464,2465,2466,2467,2468
loci_name,,,,,,,,,,,,,,,,,,,,,
chr1_3742742_3759944,2.0,11.0,4.0,2.0,2.0,0.0,0.0,11.0,0.0,0.0,...,2.0,1.0,11.0,0.0,0.0,3.0,5.0,0.0,0.0,11.0
chr1_6245958_6258969,1.0,9.0,3.0,6.0,0.0,2.0,0.0,6.0,0.0,0.0,...,2.0,1.0,10.0,1.0,2.0,0.0,3.0,1.0,0.0,3.0
chr1_8740008_8759916,0.0,10.0,10.0,1.0,0.0,4.0,3.0,4.0,1.0,1.0,...,2.0,2.0,3.0,1.0,2.0,3.0,4.0,2.0,5.0,3.0
chr1_9627926_9637875,1.0,12.0,10.0,0.0,0.0,6.0,3.0,1.0,1.0,1.0,...,1.0,2.0,2.0,3.0,3.0,4.0,3.0,1.0,6.0,6.0
chr1_9799472_9811359,2.0,14.0,11.0,0.0,0.0,5.0,3.0,2.0,1.0,1.0,...,1.0,2.0,2.0,3.0,3.0,4.0,3.0,1.0,7.0,6.0
